In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE244645"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE244645"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE244645.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE244645.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE244645.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from platelets from early NSCLC and HNSCC patients in tumour presence and tumor free samples"
!Series_summary	"Gene expression profile of platelets. In this study, we try to address the knowledge gap regarding liquid biopsy markers for early detection of non-small cell lung cancer (NSCLC) and head and neck squamous cell carcinoma (HNSCC). For that blood samples were collected in two time points, in the presence and absence of NSCLC or HNSCC. Platelets were isolated and gene expression evaluated by microarray technique."
!Series_overall_design	"Sixty nine platelet samples from NSCLC and HNSCC patients. 15 and 21 samples correspond to NSCLC and HNSCC pre-treatment, respectively, while 11 and 22 samples correspond to NSCLC and HNSCC post-treatment samples, respectively."
Sample Characteristics Dictionary:
{0: ['blood: platelet'], 1: ['condition: tumour presence', 'condition: tumour free'], 2: ['histology: Lung adenocarcinoma', 'histol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from platelets
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Lung Cancer):
# From the sample characteristics, key 2 has histology that includes "Lung adenocarcinoma"
# This indicates presence of lung cancer data
trait_row = 2

# For age:
# From the sample characteristics, key 5 has age information
age_row = 5

# For gender:
# From the sample characteristics, key 4 has gender information
gender_row = 4

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert histology information to binary Lung Cancer status"""
    if not value or value == '-':
        return None
    # If the histology contains "Lung", it's a lung cancer case (1)
    # Otherwise, it's head and neck cancer or other condition (0)
    if 'Lung' in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age string to numeric value"""
    if not value:
        return None
    # Extract the number after the colon
    try:
        # Expecting format like "age: 68"
        return int(value.split(': ')[1])
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    """Convert gender string to binary (0=female, 1=male)"""
    if not value:
        return None
    # Extract the gender value after the colon
    try:
        # Expecting format like "Sex: Male" or "Sex: Female"
        gender = value.split(': ')[1].strip().lower()
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
        else:
            return None
    except (IndexError, ValueError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # From the previous output, we have the sample characteristics dictionary
    # We need to convert this to a DataFrame for processing
    # The dictionary format is {row_index: [values_for_samples]}
    
    # For this step, we need to access the clinical_data variable which should have been
    # created in a previous step. We'll attempt to use it directly rather than loading from a file.
    
    # Let's print our findings about the clinical data
    print(f"Trait row index: {trait_row}, Age row index: {age_row}, Gender row index: {gender_row}")
    
    # Check if clinical_data is available in the environment
    try:
        # Attempt to process existing clinical_data
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the clinical features DataFrame
        preview_data = preview_df(selected_clinical_df)
        print("Preview of Selected Clinical Features:")
        print(preview_data)
        
        # Save the clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Saved clinical data to {out_clinical_data_file}")
    except NameError:
        # If clinical_data is not available, we need to wait for it
        print("Clinical data not available yet. Please run the step that generates clinical_data first.")


Trait row index: 2, Age row index: 5, Gender row index: 4
Preview of Selected Clinical Features:
{'GSM7823140': [1.0, 68.0, 1.0], 'GSM7823141': [1.0, 71.0, 0.0], 'GSM7823142': [1.0, 56.0, 1.0], 'GSM7823143': [0.0, 56.0, 1.0], 'GSM7823144': [1.0, 64.0, 0.0], 'GSM7823145': [0.0, 64.0, 0.0], 'GSM7823146': [1.0, 58.0, 0.0], 'GSM7823147': [0.0, 58.0, 0.0], 'GSM7823148': [1.0, 67.0, 1.0], 'GSM7823149': [0.0, 67.0, 1.0], 'GSM7823150': [1.0, 64.0, 1.0], 'GSM7823151': [0.0, 64.0, 1.0], 'GSM7823152': [1.0, 77.0, 0.0], 'GSM7823153': [1.0, 57.0, 1.0], 'GSM7823154': [1.0, 68.0, 0.0], 'GSM7823155': [0.0, 68.0, 0.0], 'GSM7823156': [1.0, 61.0, 0.0], 'GSM7823157': [0.0, 61.0, 0.0], 'GSM7823158': [1.0, 75.0, 1.0], 'GSM7823159': [0.0, 75.0, 1.0], 'GSM7823160': [1.0, 65.0, 0.0], 'GSM7823161': [0.0, 65.0, 0.0], 'GSM7823162': [1.0, 69.0, 0.0], 'GSM7823163': [0.0, 69.0, 0.0], 'GSM7823164': [1.0, 65.0, 0.0], 'GSM7823165': [0.0, 65.0, 0.0], 'GSM7823166': [0.0, 50.0, 0.0], 'GSM7823167': [0.0, 70.0, 1.0], 'GSM78

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1',
       'TC0100006480.hg.1', 'TC0100006483.hg.1', 'TC0100006486.hg.1',
       'TC0100006490.hg.1', 'TC0100006492.hg.1', 'TC0100006494.hg.1',
       'TC0100006497.hg.1', 'TC0100006499.hg.1', 'TC0100006501.hg.1',
       'TC0100006502.hg.1', 'TC0100006514.hg.1', 'TC0100006516.hg.1',
       'TC0100006517.hg.1', 'TC0100006524.hg.1', 'TC0100006540.hg.1',
       'TC0100006548.hg.1', 'TC0100006550.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers shown are not standard human gene symbols
# These appear to be Affymetrix or similar microarray probe identifiers (TC format with .hg.1 suffix)
# Standard human gene symbols would be like BRCA1, TP53, etc.
# These identifiers will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 10

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that contain probe IDs and gene symbols
# From the preview, we can see that 'ID' contains the probe IDs matching the gene expression data index
# The 'SPOT_ID.1' column appears to contain detailed gene information with gene symbols

# 2. Extract gene mapping using get_gene_mapping function
prob_col = 'ID'  # Column with probe IDs
gene_col = 'SPOT_ID.1'  # Column with gene information
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Print a sample of the mapping to verify
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This will handle the many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the gene expression data after mapping
print(f"Gene expression data shape after mapping: {gene_data.shape}")
# Print a few gene symbols to verify the mapping worked
print("Sample gene symbols after mapping:")
print(gene_data.index[:10])


Sample of gene mapping:
                  ID                                               Gene
0  TC0100006437.hg.1  NM_001005484 // RefSeq // Homo sapiens olfacto...
1  TC0100006476.hg.1  NM_152486 // RefSeq // Homo sapiens sterile al...
2  TC0100006479.hg.1  NM_198317 // RefSeq // Homo sapiens kelch-like...
3  TC0100006480.hg.1  NM_001160184 // RefSeq // Homo sapiens pleckst...
4  TC0100006483.hg.1  NM_005101 // RefSeq // Homo sapiens ISG15 ubiq...


Gene expression data shape after mapping: (85633, 69)
Sample gene symbols after mapping:
Index(['A-', 'A-1', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
# Save to the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
# The variable name from step 2 is selected_clinical_df, not selected_clinical_data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains platelet gene expression from lung cancer and head/neck cancer patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Data not saved.")

Shape of normalized gene data: (19975, 69)


Saved normalized gene data to ../../output/preprocess/Lung_Cancer/gene_data/GSE244645.csv
Shape of linked data: (69, 19978)


Shape of linked data after handling missing values: (69, 19978)
For the feature 'Lung_Cancer', the least common label is '1.0' with 15 occurrences. This represents 21.74% of the dataset.
The distribution of the feature 'Lung_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 61.0
  75%: 68.0
Min: 39.0
Max: 77.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 26 occurrences. This represents 37.68% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Lung_Cancer/GSE244645.csv
